In [1]:
import numpy as np
from collections import Counter

Критерий информативности разбиения. Будем использовать *энтропийный критерий*

In [2]:
def H_Entropy(list_):
    length = len(list_)
    cnt = Counter(list_)
    entropy = 0
    for var in cnt.values():
        p = var / length
        log_2 = np.log2(p)
        H = -p * log_2
        entropy += H
    
    return entropy

Information Gain (IG) - функционал качества

In [3]:
def IG(H, l, i):
    left_l = l[:i]
    right_l = l[i:]
    return H(l) - (len(left_l) / len(l)) * H(left_l) - (len(right_l) / len(l)) * H(right_l)

Строить дерево решений будем на учебном датасете iris

In [4]:
from sklearn import datasets
iris = datasets.load_iris()

In [5]:
iris.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [6]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [7]:
X = iris.data[:, : ]
Y = iris.target[:]
# iris.data[:100, : ]

In [8]:
Z = np.column_stack((X, Y))
Z

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ],
       [5.4, 3.7, 1.5, 0.2, 0. ],
       [4.8, 3.4, 1.6, 0.2, 0. ],
       [4.8, 3. , 1.4, 0.1, 0. ],
       [4.3, 3. , 1.1, 0.1, 0. ],
       [5.8, 4. , 1.2, 0.2, 0. ],
       [5.7, 4.4, 1.5, 0.4, 0. ],
       [5.4, 3.9, 1.3, 0.4, 0. ],
       [5.1, 3.5, 1.4, 0.3, 0. ],
       [5.7, 3.8, 1.7, 0.3, 0. ],
       [5.1, 3.8, 1.5, 0.3, 0. ],
       [5.4, 3.4, 1.7, 0.2, 0. ],
       [5.1, 3.7, 1.5, 0.4, 0. ],
       [4.6, 3.6, 1. , 0.2, 0. ],
       [5.1, 3.3, 1.7, 0.5, 0. ],
       [4.8, 3.4, 1.9, 0.2, 0. ],
       [5. , 3. , 1.6, 0.2, 0. ],
       [5. , 3.4, 1.6, 0.4, 0. ],
       [5.2, 3.5, 1.5, 0.2, 0. ],
       [5.2, 3.4, 1.4, 0.2, 0. ],
       [4.7, 3

В функции optima(matrix) работаю только с 1 признаком в векторе признаков:  
['sepal length (cm)',  
 'sepal width (cm)',  
 'petal length (cm)',  
 'petal width (cm)']    
 те с sepal length (cm).  
 Сортирую матрицу по sepal length (cm), остальные столбцы не изменяю.  
 После сортировки выделяю в отдельный список столбец с метками. И дальше работаю с ним.  
 Применяю IG и ищу лучшее место для разделения.  
 Запоминаю [i_max, max_IG]

In [9]:
def optima(matrix):
    matrix.view('f8,f8,f8,f8,f8').sort(order=['f0'], axis=0)
    column_target = []
    for num in matrix:
        column_target.append(num[-1])
            
    max_IG = IG(H_Entropy, column_target, 0)
    i_max = len(column_target)
    for i in range(len(column_target) - 1):
        IG_ = IG(H_Entropy, column_target, i + 1)
        if IG_ >= max_IG:
            max_IG = IG_
            i_max = i + 1
    return [i_max, max_IG]

По max_IG определяю лист или узел, и продолжаю рекурсивно.

In [12]:
def tree(matrix):
    i_max, max_IG = optima(matrix)            
    if max_IG == 0:
        return print(f'Лист \n{matrix[:i_max + 1,:]}\n')
    else:
        tree(matrix[:i_max, :])
        tree(matrix[i_max:, :])

In [13]:
tree(Z)

Лист 
[[4.3 3.  1.1 0.1 0. ]
 [4.4 2.9 1.4 0.2 0. ]
 [4.4 3.  1.3 0.2 0. ]
 [4.4 3.2 1.3 0.2 0. ]
 [4.5 2.3 1.3 0.3 0. ]
 [4.6 3.1 1.5 0.2 0. ]
 [4.6 3.2 1.4 0.2 0. ]
 [4.6 3.4 1.4 0.3 0. ]
 [4.6 3.6 1.  0.2 0. ]
 [4.7 3.2 1.3 0.2 0. ]
 [4.7 3.2 1.6 0.2 0. ]
 [4.8 3.  1.4 0.1 0. ]
 [4.8 3.  1.4 0.3 0. ]
 [4.8 3.1 1.6 0.2 0. ]
 [4.8 3.4 1.6 0.2 0. ]
 [4.8 3.4 1.9 0.2 0. ]]

Лист 
[[4.9 2.4 3.3 1.  1. ]]

Лист 
[[4.9 2.5 4.5 1.7 2. ]]

Лист 
[[4.9 3.  1.4 0.2 0. ]
 [4.9 3.1 1.5 0.1 0. ]
 [4.9 3.1 1.5 0.2 0. ]
 [4.9 3.6 1.4 0.1 0. ]]

Лист 
[[5.  2.  3.5 1.  1. ]
 [5.  2.3 3.3 1.  1. ]]

Лист 
[[5.  3.  1.6 0.2 0. ]
 [5.  3.2 1.2 0.2 0. ]
 [5.  3.3 1.4 0.2 0. ]
 [5.  3.4 1.5 0.2 0. ]
 [5.  3.4 1.6 0.4 0. ]
 [5.  3.5 1.3 0.3 0. ]
 [5.  3.5 1.6 0.6 0. ]
 [5.  3.6 1.4 0.2 0. ]]

Лист 
[[5.1 2.5 3.  1.1 1. ]]

Лист 
[[5.1 3.3 1.7 0.5 0. ]
 [5.1 3.4 1.5 0.2 0. ]
 [5.1 3.5 1.4 0.2 0. ]
 [5.1 3.5 1.4 0.3 0. ]
 [5.1 3.7 1.5 0.4 0. ]
 [5.1 3.8 1.5 0.3 0. ]
 [5.1 3.8 1.6 0.2 0. ]
 [5.1 3.8 1.9 0.4 

Видно, что мое дерево переобучилось, тк стало выделять каждый лист.

**Что нужно доделать:**  
1) Критерий остановки, чтобы дерево не переобучалось.  
2) В данном решении я работаю только с одним признаком из вектора признаков их всего 4. 
 - Как его реализовать? Осуществить обычный перебор вначале с 1 признаком посмотреть его IG потом со 2 и так до 4 и выбрать лучшее разбиение по IG?  

3) Необходимо отдельно запоминать правила разбиения в каждом узле, чтобы потом предсказывать класс по новым данным.  
Также отмечу, что в учебном датасете отсутсвуют пропуски, что тоже необходимо учитывать. 